In [2]:
import re
import os
import pandas as pd
import numpy as np

if os.name == 'posix':
    import pathlib
    pathlib.WindowsPath = pathlib.PosixPath

In [2]:
pfcpaths = pd.read_pickle('/Users/lukearend/phd/kfx/ref/PFC_lfp_paths.file')
hpcpaths = pd.read_pickle('/Users/lukearend/phd/kfx/ref/HPC_lfp_paths.file')

In [3]:
for pfcpath, hpcpath in zip(pfcpaths, hpcpaths):
    assert str(pfcpath).replace('imec0', 'imec1') == str(hpcpath)

PFC paths and HPC paths are all exactly identical, except that PFC contains the substring `imec0` and HPC contains substring `imec1`. These are the identifiers for the two NeuroPixels probes used.

In [4]:
from datetime import datetime

def sessinfo(hpcpath):
    probedir, file = os.path.split(hpcpath)
    sessdir, probedir = os.path.split(probedir)
    _, sessdir = os.path.split(sessdir)
    indiv = re.search(r'(M\d{3})_', sessdir)[1]
    drug = re.search(r'_([a-zA-Z0-9-]*)_[m]*PFC', sessdir)[1]
    doses = re.search(r'HPC_([\w]*mpk)', sessdir)[1]
    doses = tuple(int(n) for n in doses.strip('mpk').split('_'))
    date = re.search(r'(2022-\d{2}-\d{2})', sessdir)[1]
    date = datetime.fromisoformat(date)
    path = os.path.join(sessdir, probedir.replace('_imec1', ''))
    return {
        'indiv': indiv,
        'date': date,
        'drug': drug,
        'doses': doses,
        'path': path,
    }

In [5]:
df = pd.DataFrame([sessinfo(path) for path in hpcpaths])
assert np.all(df == df.sort_values('date'))

The `sessnum` in Gino's code is an index going from 0 to 35 in order of session recording date.

In [6]:
df.index.name = 'recording'
df.head()

,indiv,date,drug,doses,path
recording,,,,,
0,M015,2022-07-27,SAL,"(0, 0, 0)",2022-07-27-07-41-00_M015_SAL_PFC_HPC_0_0_0mpk/...
1,M016,2022-07-28,SAL,"(0, 0, 0)",2022-07-28-13-19-00_M016_SAL_PFC_HPC_0_0_0mpk/...
2,M015,2022-08-01,RSK,"(3, 10, 30)",2022-08-01_04-30-00_M015_RSK_mPFC_HPC_3_10_30m...
3,M015,2022-08-03,RSK,"(3, 10, 30)",2022-08-03_12-45-00_M015_RSK_mPFC_HPC_3_10_30m...
4,M016,2022-08-04,RSK,"(3, 10, 30)",2022-08-04_03-45-00_M016_RSK_mPFC_HPC_3_10_30m...


In [7]:
df.to_csv('/Users/lukearend/phd/kfx/ref/sessions.csv')

In [4]:
pd.read_csv('/Users/lukearend/phd/kfx/ref/sessions.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/lukearend/phd/kfx/ref/sessions.csv'

In [1]:
df

NameError: name 'df' is not defined